<div style="text-align: center; background-color: #0A6EBD; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
    FIT-HCMUS, VNU-HCM 
    <br>
    Introduction To Data Science 
    <br>
    Final project 📌
</div>

<div style="text-align: center; background-color: #5A96E3; font-family: 'Trebuchet MS', Arial, sans-serif; color: white; padding: 20px; font-size: 40px; font-weight: bold; border-radius: 0 0 0 0; box-shadow: 0px 6px 8px rgba(0, 0, 0, 0.2);">
  Stage 04 - Feature encoding and scaling 📌
</div>

# Import library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import skew
from sklearn.preprocessing import StandardScaler,RobustScaler

In [2]:
df = pd.read_csv('../../data/processed/processed_fb_data.csv')

In [3]:
df.head()

,Name,Position,PreferredFoot,Season,Age,Squad,Country,Comp,LgRank,MP,...,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90,Injury,from,until,Days,Games missed
0,William Saliba,DF,Right,2018-2019,17,Saint-Étienne,FRA,Ligue 1,4,16,...,0.020000,0.000000,0.020000,0.020000,0.020000,Hamstring injury,2019-05-23,2019-09-28,128.0,22.0
1,William Saliba,DF,Right,2019-2020,18,Saint-Étienne,FRA,Ligue 1,17,12,...,0.020000,0.010000,0.030000,0.020000,0.030000,Metatarsal fracture,2019-11-04,2020-01-20,77.0,19.0
2,William Saliba,DF,Right,2020-2021,19,Nice,FRA,Ligue 1,9,20,...,0.040000,0.000000,0.040000,0.040000,0.040000,NaN,NaN,NaN,NaN,NaN
3,William Saliba,DF,Right,2020-2021,19,Arsenal,ENG,Jr. PL2 -- Div. 1,10,6,...,0.058096,0.079837,0.137923,0.056746,0.136612,NaN,NaN,NaN,NaN,NaN
4,William Saliba,DF,Right,2021-2022,20,Marseille,FRA,Ligue 1,2,36,...,0.050000,0.010000,0.060000,0.050000,0.060000,NaN,NaN,NaN,NaN,NaN


# Preprocessing 

The reason to drop these columns : 
- The Squad column don't provide much information bacause the dataset has LgRank whose meaning is the rank of the squad in the league . The teams in the top of table usually have better stats than the teams in the opposite side.
- The Country,Comp column just show the name of country,league of the player's club . Although each country can have different style of playing football such as Brazil is famous for attacking, Italy is known as good defending ,...   ,it is not enough information to judge all countries ,leagues
- These 2 columns : from , until show the begining and ending of the injury . The dataset has more informative columns is Days (counting the number of days player missed)


In [4]:
df.drop(['Squad','Country','Comp','from','until'],inplace=True,axis = 'columns')

In [5]:
#Re-define the meaning of Injury column : Injury column is used to identify that the player has the injury or not
df['Injury'] = df['Injury'].apply(lambda x : 0 if x is np.nan else 1)

In [6]:
#Fill missing values
df['Days'].fillna(0,inplace=True)
df['Games missed'].fillna(0,inplace=True)

Calculate all performance stats and injury stats according to each season of each player

In [7]:
#create a dictionary for each column in performance and injury stats with a function aggregate 
agg_dict = {}
for col in df.columns[6:]:
    if '90' in col:
        agg_dict[col]='mean'
    else:
        agg_dict[col] = 'sum'

In [8]:
info_df = df.groupby(['Name','Season']).agg(agg_dict)
info_df.head()

MP  Starts    Min   90s  Gls  Ast  G+A  G-PK  PK  \
Name            Season                                                         
Aaron Cresswell 2014-2015   38      38   3420  38.0    2    4    6     2   0   
                2015-2016   37      37   3314  36.8    2    4    6     2   0   
                2016-2017   52      48   4232  23.5    0    4    4     0   0   
                2017-2018   36      35   3069  34.1    1    3    4     1   0   
                2018-2019  140     126  11123  17.7    0    7    7     0   0   

                           PKatt  ...  G-PK/90  G+A-PK/90     xG/90    xAG/90  \
Name            Season            ...                                           
Aaron Cresswell 2014-2015      0  ...     0.05       0.16  0.058096  0.079837   
                2015-2016      0  ...     0.05       0.16  0.058096  0.079837   
                2016-2017      0  ...     0.00       0.09  0.058096  0.079837   
                2017-2018      0  ...     0.03       0.12  0.020000  0.090000   
                2018-2019      0  ...     0.00       0.06  0.030000  0.060000   

                           xG+xAG/90   npxG/90  npxG+xAG/90  Injury  Days  \
Name            Season                                                      
Aaron Cresswell 2014-2015   0.137923  0.056746     0.136612       0   0.0   
                2015-2016   0.137923  0.056746     0.136612       0   0.0   
                2016-2017   0.137923  0.056746     0.136612       2  95.0   
                2017-2018   0.120000  0.020000     0.120000       0   0.0   
                2018-2019   0.090000  0.030000     0.090000       7  87.0   

                           Games missed  
Name            Season                   
Aaron Cresswell 2014-2015           0.0  
                2015-2016           0.0  
                2016-2017          19.0  
                2017-2018           0.0  
                2018-2019          10.0  

[5 rows x 32 columns]

In [9]:
df.drop(df.columns[6:],axis='columns',inplace=True)

In [10]:
#merge the info_df to the main df by 2 column name and season
df = df.merge(right=info_df,how='inner',on=['Name','Season'])

In [11]:
#drop duplicate column because the df has some duplicate name and season when player play in 2 different team in 1 season
df.drop_duplicates(keep='first',inplace=True)

After being used to calculate the performance and injury stats, the Name and Season columns become unnecessary so we decide to drop these column

In [12]:
df.drop(['Name','Season'],axis='columns',inplace=True)

In [13]:
df.head()

,Position,PreferredFoot,Age,LgRank,MP,Starts,Min,90s,Gls,Ast,...,G-PK/90,G+A-PK/90,xG/90,xAG/90,xG+xAG/90,npxG/90,npxG+xAG/90,Injury,Days,Games missed
0,DF,Right,17,4,16,13,1277,14.2,0,0,...,0.000,0.000,0.020000,0.000000,0.020000,0.020000,0.020000,1,128.0,22.0
1,DF,Right,18,17,12,11,992,11.0,0,0,...,0.000,0.000,0.020000,0.010000,0.030000,0.020000,0.030000,1,77.0,19.0
2,DF,Right,19,9,26,26,2326,12.9,1,0,...,0.025,0.025,0.049048,0.039919,0.088962,0.048373,0.088306,0,0.0,0.0
3,DF,Right,19,10,26,26,2326,12.9,1,0,...,0.025,0.025,0.049048,0.039919,0.088962,0.048373,0.088306,0,0.0,0.0
4,DF,Right,20,2,36,36,3240,36.0,0,0,...,0.000,0.000,0.050000,0.010000,0.060000,0.050000,0.060000,0,0.0,0.0


In [14]:
cate_col = df.select_dtypes(include=object).columns
num_col = df.select_dtypes(exclude=object).columns

# One-hot encoding for categorical column

In [15]:
#Find categorical column in the dataframe
cate_col

Index(['Position', 'PreferredFoot'], dtype='object')

In [16]:
#One-hot encoding for cate_col
df = pd.get_dummies(df, columns=cate_col)
df

,Age,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,...,npxG+xAG/90,Injury,Days,Games missed,Position_DF,Position_FW,Position_MF,PreferredFoot_ Both,PreferredFoot_ Left,PreferredFoot_ Right
0,17,4,16,13,1277,14.2,0,0,0,0,...,0.020000,1,128.0,22.0,True,False,False,False,False,True
1,18,17,12,11,992,11.0,0,0,0,0,...,0.030000,1,77.0,19.0,True,False,False,False,False,True
2,19,9,26,26,2326,12.9,1,0,1,1,...,0.088306,0,0.0,0.0,True,False,False,False,False,True
3,19,10,26,26,2326,12.9,1,0,1,1,...,0.088306,0,0.0,0.0,True,False,False,False,False,True
4,20,2,36,36,3240,36.0,0,0,0,0,...,0.060000,0,0.0,0.0,True,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4566,20,14,30,30,2700,30.0,2,0,2,2,...,0.136612,0,0.0,0.0,True,False,False,False,False,True
4567,21,10,26,22,1946,21.6,0,0,0,0,...,0.136612,0,0.0,0.0,True,False,False,False,False,True
4568,22,3,26,25,2234,24.8,4,0,4,4,...,0.136612,0,0.0,0.0,True,False,False,False,False,True
4569,23,2,33,32,2790,15.5,4,1,5,4,...,0.128306,0,0.0,0.0,True,False,False,False,False,True


In [17]:
#convert from bool type to int type 
bool_col = df.select_dtypes(include=bool)
for col in bool_col:
    df[col] =df[col].astype(np.int64)

In [18]:
df.reset_index(inplace=True,drop=True)

In [19]:
df

,Age,LgRank,MP,Starts,Min,90s,Gls,Ast,G+A,G-PK,...,npxG+xAG/90,Injury,Days,Games missed,Position_DF,Position_FW,Position_MF,PreferredFoot_ Both,PreferredFoot_ Left,PreferredFoot_ Right
0,17,4,16,13,1277,14.2,0,0,0,0,...,0.020000,1,128.0,22.0,1,0,0,0,0,1
1,18,17,12,11,992,11.0,0,0,0,0,...,0.030000,1,77.0,19.0,1,0,0,0,0,1
2,19,9,26,26,2326,12.9,1,0,1,1,...,0.088306,0,0.0,0.0,1,0,0,0,0,1
3,19,10,26,26,2326,12.9,1,0,1,1,...,0.088306,0,0.0,0.0,1,0,0,0,0,1
4,20,2,36,36,3240,36.0,0,0,0,0,...,0.060000,0,0.0,0.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3315,20,14,30,30,2700,30.0,2,0,2,2,...,0.136612,0,0.0,0.0,1,0,0,0,0,1
3316,21,10,26,22,1946,21.6,0,0,0,0,...,0.136612,0,0.0,0.0,1,0,0,0,0,1
3317,22,3,26,25,2234,24.8,4,0,4,4,...,0.136612,0,0.0,0.0,1,0,0,0,0,1
3318,23,2,33,32,2790,15.5,4,1,5,4,...,0.128306,0,0.0,0.0,1,0,0,0,0,1


# Feature scaling for numerical columns

In [20]:
#Find numerical columns
num_col

Index(['Age', 'LgRank', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A',
       'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG',
       'PrgC', 'PrgP', 'PrgR', 'Gls/90', 'Ast/90', 'G+A/90', 'G-PK/90',
       'G+A-PK/90', 'xG/90', 'xAG/90', 'xG+xAG/90', 'npxG/90', 'npxG+xAG/90',
       'Injury', 'Days', 'Games missed'],
      dtype='object')

Calculate the percentage of outliers in each column

In [21]:
def count_outlier(df,num_col):
    outlier_dict ={}
    for col in num_col:
        Q1 = np.percentile(df[col],25)
        Q3 = np.percentile(df[col],75)
        iqr = Q3 - Q1
        outliers = ((df[col] < (Q1 - 1.5 * iqr)) | (df[col] > (Q3 + 1.5 * iqr)))
        outlier_dict[col] = outliers.sum() / len(df[col])
    return outlier_dict

In [22]:
outlier_dict =count_outlier(df,num_col)
outlier_df = pd.DataFrame(list(outlier_dict.items()), columns=['Column', 'Outlier Percentage'])
outlier_df['Skew'] = [skew(df[col],bias=False) for col in num_col]
outlier_df.sort_values('Outlier Percentage',ascending=False)

,Column,Outlier Percentage,Skew
32,Days,0.139458,3.182785
33,Games missed,0.125301,3.556875
11,PKatt,0.121386,7.634470
13,CrdR,0.111747,4.881115
10,PK,0.107229,8.136370
16,xAG,0.090060,3.732981
17,npxG+xAG,0.087048,3.982257
15,npxG,0.085843,5.088250
14,xG,0.085542,5.100006
20,PrgR,0.080723,3.165261


With the column having high outliers percentages, we apply RobustScaler to scale the data because the large number of outliers can affect the dataset and the accuracy of model in the next section

In [23]:
# find the column with high percentage of outliers ( Percentage >= 5%)
highoutlier_col = outlier_df.loc[outlier_df['Outlier Percentage']>=0.05]['Column'].to_list()
print(highoutlier_col)

['MP', 'Starts', 'Min', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR', 'Gls/90', 'Ast/90', 'G-PK/90', 'Days', 'Games missed']


In [24]:
#Apply RobustScaler for these columns
scaler = RobustScaler()
scaler.fit(df[highoutlier_col])
scaled_data = scaler.transform(df[highoutlier_col])
df[highoutlier_col] = scaled_data

With the column having positive skewness ,we apply Cube root transformation to reduce the skewness of data . THe reason why we don't choose Square Root transformation is that the data in many columns has zero 

In [25]:
#find the column with positive skewness 
pos_skew_col = outlier_df.loc[outlier_df['Skew']>=outlier_df['Skew'].median()]['Column'].to_list()
print(pos_skew_col)

['Gls', 'G+A', 'G-PK', 'PK', 'PKatt', 'CrdR', 'xG', 'npxG', 'PrgC', 'Ast/90', 'G+A/90', 'G+A-PK/90', 'xG/90', 'xAG/90', 'xG+xAG/90', 'npxG/90', 'npxG+xAG/90']


In [26]:
#Apply Cube Root Transformation for these columns
for col in pos_skew_col:
    df[col] = df[col].pow(1/3)

# Save to csv file

In [27]:
df.to_csv("../../data/processed/scaled_encoded_data.csv", index=False)